# LangChain Expression Language (LCEL)

https://python.langchain.com/docs/expression_language/get_started

> LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.



## Basic example: prompt + model + output

In [2]:
# Basic example: prompt + model + output

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Tell me a short joke about the difference between Japanese and German on {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [3]:
chain.invoke({"topic": "football"})

'Why do Japanese football teams always beat German ones? \n\nBecause the Germans are too busy trying to perfect their precision passing while the Japanese are just focused on scoring goals!'

## RAG example

In [1]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstores = DocArrayInMemorySearch.from_texts(
    ["Manuel Neuer works for Bayern Munich.", "Bayern Munich's stadium is Allianz Arena."],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstores.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

/Users/takuro/Developer/cha9ro/llm-snipets/.venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [2]:
chain.invoke("What is Manuel Neuer's home stadium?")

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [0.01859499..., -0.01637975235394259]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [0.01859499..., -0.01637975235394259]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing